# Project - Cricket Anaylyst AI Assistant

Cricket Analyst AI Assistant is an intelligent tool that analyzes cricket data to compare players, evaluate performances across formats, and provide insightful statistics. It processes historical and recent match data to deliver easy-to-understand summaries, helping fans, analysts, and coaches make informed decisions.

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import speech_recognition as sr
import pandas as pd


In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = """
You are a Cricket Analyst AI with deep knowledge of cricket statistics and match analysis.
When comparing players, call the `analyze_cricket` tool to get factual data before answering.
"""

In [ ]:
# Sample cricket stats as a list of dicts
cricket_data = [
    {"Player": "Virat Kohli", "Format": "ODI", "Year": 2023, "Runs": 1377, "Matches": 27, "Average": 57.37, "StrikeRate": 93.21},
    {"Player": "Virat Kohli", "Format": "ODI", "Year": 2022, "Runs": 765, "Matches": 20, "Average": 42.50, "StrikeRate": 88.40},
    {"Player": "Virat Kohli", "Format": "ODI", "Year": 2021, "Runs": 560, "Matches": 15, "Average": 40.00, "StrikeRate": 90.10},
    {"Player": "Babar Azam", "Format": "ODI", "Year": 2023, "Runs": 1454, "Matches": 26, "Average": 62.00, "StrikeRate": 89.50},
    {"Player": "Babar Azam", "Format": "ODI", "Year": 2022, "Runs": 1198, "Matches": 18, "Average": 66.55, "StrikeRate": 92.00},
    {"Player": "Babar Azam", "Format": "ODI", "Year": 2021, "Runs": 949, "Matches": 15, "Average": 67.78, "StrikeRate": 90.50},
    {"Player": "Joe Root", "Format": "Test", "Year": 2025, "Runs": 949, "Matches": 15, "Average": 69.78, "StrikeRate": 95.50},
    {"Player": "Joe Root", "Format": "Test", "Year": 2024, "Runs": 2025, "Matches": 22, "Average": 68.78, "StrikeRate": 90.50},
    {"Player": "Harry Brook", "Format": "Test", "Year": 2025, "Runs": 1056, "Matches": 16, "Average": 67.78, "StrikeRate": 95.50},
    {"Player": "Harry Brook", "Format": "Test", "Year": 2024, "Runs": 2200, "Matches": 21, "Average": 71.78, "StrikeRate": 98.50},

]

## Tools

Tools starts from here. 
For this notebook, I have just wrote one Tool, you can add multiple tools for your agent.

In [ ]:
def analyze_cricket(data_list, player1, match_format="ODI", years=3):
    """
    Return cricket players' performances using an in-memory list of dicts.
    """
    print("Tool 'analyze_cricket' is called")
    df = pd.DataFrame(data_list)
    latest_year = df['Year'].max()
    min_year = latest_year - years + 1

    filtered = df[
        (df['Format'].str.upper() == match_format.upper()) &
        (df['Year'] >= min_year) &
        (df['Player'].isin([player1]))
    ]
    if filtered.empty:
        return {"error": f"No data found for {player1} in {match_format} for last {years} years."}

    summary = filtered.groupby("Player").agg({
        "Matches": "sum",
        "Runs": "sum",
        "Average": "mean",
        "StrikeRate": "mean"
    }).round(2)

    return summary.reset_index().to_dict(orient="records")

In [ ]:
# Example usage:
result = analyze_cricket(cricket_data, "Virat Kohli", "ODI", 3)
print(result)

In [ ]:
# Tool definition 
analyze_cricket_functions = {
        "name": "analyze_cricket",
        "description": "Compare two cricket players' performances over the last N years.",
        "parameters": {
            "type": "object",
            "properties": {
                "player1": {"type": "string", "description": "Name of first player"},
                # "player2": {"type": "string", "description": "Name of second player"},
                "match_format": {"type": "string", "enum": ["ODI", "Test", "T20"], "description": "Format of the match"},
                "years": {"type": "integer", "description": "Number of years to compare"}
            },
            "required": ["player1"]
        }
    }

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": analyze_cricket_functions}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        for tool_call in message.tool_calls: 
            response, player1= handle_tool_call(tool_call)
            messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(tool_call):
    # tool_call = message.tool_calls[0]
    # print("tool_call.id", tool_call.id)
    arguments = json.loads(tool_call.function.arguments)
    print("arguments", arguments)
    player1 = arguments.get('player1')
    # player2 = arguments.get('player2')
    match_format = arguments.get('match_format', 'ODI')
    years = arguments.get('years', 3)
    result = analyze_cricket(cricket_data, player1, match_format, years)
    print("result from analyze_cricket function: ", tool_call.id, result)
    response = {
        "role": "tool",
        "content": json.dumps(result),
        "tool_call_id": tool_call.id
    }
    return response, player1

# Sample User prompt

1. ### Compare Babar and Virat in ODI matches over the last 3 years.
Here 
Player1 is Babar
Player2 is Virat
match_format is ODI
years is 3


2. ### can you please give me the comparison of Virat and babar?
Here, you are not provided the info on the format and number of years. In this case, the function will pick the default values for the match format, which is ODI, and the years, which is 3.


3. ### Compare Rizwan and Babar in ODI Matches over the last years.
The given data is not available in the above data list. 


In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4 cents - don't go crazy with images!

In [ ]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(player_names):
    if len(player_names) <2 or len(player_names) > 2:
        return None
    player1 = player_names[0]
    player2 = player_names[1]
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a comparison of {player1} and {player2}, showing their country flags and bowling or batting style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist(["Babar", "root"])
display(image)

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
talker("Well, hi there") # For testing purposes

# Our Agent Framework

The term 'Agentic AI' and Agentization is an umbrella term that refers to a number of techniques, such as:

1. Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
2. The ability for LLMs to use Tools to give them additional capabilities
3. The 'Agent Environment' which allows Agents to collaborate
4. An LLM can act as the Planner, dividing bigger tasks into smaller ones for the specialists
5. The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory


In [ ]:
def chat(history, image_choice):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        player_names = []
        for tool_call in message.tool_calls:
            response, player1= handle_tool_call(tool_call)
            player_names.append(player1)
            messages.append(response)
        if image_choice.lower() == 'yes':
            image = artist(player_names)
        else:
            print("Image value is NO", image_choice)
            
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    gr.Markdown("### 🏏 Cricket Analyst AI Assistant")
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)

    with gr.Row():
        image_dropdown = gr.Dropdown(
        choices=["Yes", "No"],
        label="Do you want image?"
    )
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history, image_choice):
        history += [{"role": "user", "content": message}]
        return "", history, image_choice

    entry.submit(
        do_entry, 
        inputs=[entry, chatbot, image_dropdown], 
        outputs=[entry, chatbot, image_dropdown]
    ).then(
        chat, 
        inputs=[chatbot, image_dropdown], 
        outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)